In [1]:
%reload_ext autoreload
%autoreload 3
import os

import xarray as xr
from pathlib import Path
from tqdm.auto import tqdm
import torch
import torch.nn.functional as F

from sklearn.preprocessing import LabelEncoder

from torch import nn
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from dvclive.lightning import DVCLiveLogger

from src.data.julia2018 import Julia2018DataModule
from src.acnets import ACNets


# force running in the project directory (e.g., via nbconvert)
if 'notebooks' in os.getcwd():
    %cd ..

In [2]:
datamodule = Julia2018DataModule(segment_size=31, split_on='sequence', batch_size=96, shuffle=False)
datamodule.prepare_data()
n_subjects = len(datamodule.subjects)
n_inputs = datamodule.n_features

In [3]:

n_features = 64
hidden_size = 32

model = ACNets(n_inputs, n_features, hidden_size, n_subjects)
# model = torch.compile(model)  # is not supported on GTX 1070

trainer = pl.Trainer(
    max_epochs=1000, accelerator='cpu', log_every_n_steps=1,
    logger=[#DVCLiveLogger(save_dvc_exp=True, report='md'),
            TensorBoardLogger('.')]
    )
trainer.fit(model, datamodule=datamodule)


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/morteza/micromamba/envs/acnets-multihead/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(

  | Name              | Type             | Params
-------------------------------------------------------
0 | feature_extractor | FeatureExtractor | 159 K 
1 | auto_encoder      | SeqAutoEncoder   | 41.8 K
2 | head_cls          | Sequential       | 562   
3 | head_subj_cls     | Linear           | 1.1 K 
-------------------------------------------------------
203 K     Trainable params
0         Non-trainable params
203 K     Total params
0.813     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[2] 64 32 torch.Size([32, 30, 64]) torch.Size([1, 32, 32])


RuntimeError: Expected hidden[0] size (1, 32, 64), got [1, 32, 32]